In [1]:
import csv
import numpy as np
import os
import pandas as pd
import time
import pybel

In [6]:
'''2017.4.20. gold에 포함되지 않는 indep drug와 gold drug간의 similarity 정보도 필요함.'''
indep_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/new_predict/3_indep_learning_asso.tsv")
print len(indep_asso['drug_id'].drop_duplicates())
gold_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/new_predict/1_gold_learning_asso.tsv")
print len(gold_asso['drug_id'].drop_duplicates())

71
434


In [7]:
gold_dr_id = gold_asso['drug_id'].drop_duplicates()
indep_dr_id = indep_asso['drug_id'].drop_duplicates()

In [9]:
drugbank = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/drug_set.txt", header=None)
drugbank = drugbank.rename(columns = {0:'DrugBank_ID', 1:'Drug_Name', 2:'InChI', 3:'SMILES', 4:'State'})
drugbank.set_index(['DrugBank_ID'], inplace = True)
drugbank.head(1)

,Drug_Name,InChI,SMILES,State
DrugBank_ID,,,,
DB00006,Bivalirudin,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,approved


In [14]:
indep_sim_mat = pd.DataFrame(columns = gold_dr_id.tolist(), index = indep_dr_id.tolist())
indep_sim_mat = indep_sim_mat.fillna(0.0)
indep_sim_mat.head(1)

,DB00659,DB01193,DB00316,DB00819,DB00210,DB00640,DB00240,DB00630,DB00346,DB09026,...,DB00162,DB00126,DB02546,DB00682,DB00549,DB00744,DB00246,DB00399,DB00315,DB00909
DB01200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
for index, row in indep_sim_mat.iterrows():
    indep_smi = drugbank.filter(items = [index], axis = 0)['SMILES'].tolist()[0]
    for dr_id in gold_dr_id:
        gold_smi = drugbank.filter(items = [dr_id], axis = 0)['SMILES'].tolist()[0]
        
        smiles = [indep_smi, gold_smi]
        mol = [pybel.readstring('smi',x) for x in smiles]
        fps = [x.calcfp(fptype='FP2') for x in mol]
        sim = fps[0] | fps[1]
        indep_sim_mat.set_value(index, dr_id, sim)  

In [34]:
indep_sim_mat.isnull().any().any()

False

In [36]:
indep_sim_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/2_2_2_indep_drug_gold_drug_str_sim_mat.tsv", sep='\t')
indep_sim_mat.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/2_2_2_indep_drug_gold_drug_str_sim_mat.tsv", sep='\t')

In [ ]:
'''2017.4.20. 끝'''

In [3]:
# indep의 structure sim descriptor 
indep_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/2_indep_drug_drug_str_sim_mat.tsv", index_col = 0)

In [5]:
print len(indep_sim)
indep_sim.head()

71


,DB00659,DB01193,DB00316,DB00819,DB00210,DB00640,DB00240,DB00630,DB00346,DB09026,...,DB00162,DB00126,DB02546,DB00682,DB00549,DB00744,DB00246,DB00399,DB00315,DB00909
DB01200,0.084559,0.190789,0.088889,0.106109,0.145833,0.176282,0.107639,0.045775,0.271523,0.232082,...,0.070111,0.121107,0.130435,0.133333,0.304217,0.145215,0.253968,0.077703,0.352273,0.116505
DB00575,0.059406,0.129870,0.232558,0.061224,0.059701,0.071856,0.031496,0.028302,0.107345,0.074534,...,0.009901,0.038168,0.215686,0.076433,0.118721,0.104167,0.189655,0.056452,0.134228,0.075342
DB01037,0.147059,0.218487,0.184615,0.058333,0.241758,0.079137,0.040816,0.081081,0.128378,0.252174,...,0.042857,0.080808,0.187500,0.186441,0.148936,0.192661,0.202703,0.051546,0.284404,0.113043
DB00280,0.121739,0.240506,0.174312,0.078788,0.179856,0.129213,0.069930,0.065041,0.204420,0.258065,...,0.033333,0.104895,0.226891,0.201258,0.219178,0.198675,0.279330,0.108696,0.299320,0.140127
DB00343,0.121693,0.322275,0.198864,0.082988,0.216749,0.154472,0.067873,0.097436,0.270042,0.263636,...,0.035000,0.115741,0.211640,0.165957,0.263538,0.133047,0.294606,0.113208,0.239819,0.144105


In [2]:
indep_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/9_2_indep_learning_association.tsv")
print len(indep_asso)
print len(indep_asso[indep_asso.association == 1]) # positive
print len(indep_asso[indep_asso.association == 0]) # negative
print len(indep_asso['drug'].drop_duplicates())
print len(indep_asso['disease'].drop_duplicates())
indep_asso.head(1)

2555
93
2462
73
35


,drug,disease,association
0,DB00125,102300,0


In [3]:
drugbank = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/drug_set.txt", header=None)
drugbank = drugbank.rename(columns = {0:'DrugBank_ID', 1:'Drug_Name', 2:'InChI', 3:'SMILES', 4:'State'})
drugbank.head(1)

,DrugBank_ID,Drug_Name,InChI,SMILES,State
0,DB00006,Bivalirudin,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,approved


In [4]:
indep_drug_id = indep_asso['drug'].drop_duplicates()
#indep_drug의 순서대로 drugbank를 정렬하기.
df_indep_drug_smiles = drugbank[drugbank['DrugBank_ID'].isin(indep_drug_id)]
sorter = indep_drug_id.tolist()
df_indep_drug_smiles.DrugBank_ID = df_indep_drug_smiles.DrugBank_ID.astype("category")
df_indep_drug_smiles.DrugBank_ID.cat.set_categories(sorter, inplace=True)
df_indep_drug_smiles = df_indep_drug_smiles.sort_values(["DrugBank_ID"])
print len(df_indep_drug_smiles)
df_indep_drug_smiles.head(1) # drug bank id로 중복체크 했을 때 중복 없었음.

71


/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,DrugBank_ID,Drug_Name,InChI,SMILES,State
17,DB00125,L-Arginine,InChI=1S/C6H14N4O2/c7-4(5(11)12)2-1-3-10-6(8)9...,N[C@@H](CCCNC(N)=N)C(O)=O,approved


In [5]:
new_asso = indep_asso[indep_asso['drug'].isin(df_indep_drug_smiles['DrugBank_ID'])]

In [6]:
new_asso.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association_indep.tsv",sep='\t', index=False)
new_asso.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association_indep.tsv", sindex=False)

In [8]:
import pybel
df_sim = pd.DataFrame()
drug_id = df_indep_drug_smiles['DrugBank_ID'].tolist()
for i in drug_id:
    smi1 = df_indep_drug_smiles[df_indep_drug_smiles.DrugBank_ID == i]['SMILES'].tolist()[0]
    for j in drug_id:
        smi2 = df_indep_drug_smiles[df_indep_drug_smiles.DrugBank_ID == j]['SMILES'].tolist()[0]
        
        smiles = [smi1, smi2]
        mol = [pybel.readstring('smi',x) for x in smiles]
        fps = [x.calcfp(fptype='FP2') for x in mol]
        sim = fps[0] | fps[1]
        
        temp_df = pd.DataFrame([[i, j, sim]], columns = ['DrugBank_ID1','DrugBank_ID2', 'CheSim'])
        df_sim = df_sim.append(temp_df)

In [10]:
df_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_structure.tsv", sep='\t', index=False)
df_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_structure.tsv", sep='\t', index=False)

In [ ]:
df_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_structure.tsv")

In [ ]:
##### phenotypic similaritied

In [ ]:
'''2017.4.21. indep drug X gold drug 간의 phenotype similarity가 필요'''

In [8]:
indep_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/new_predict/3_indep_learning_asso.tsv")
print len(indep_asso['drug_id'].drop_duplicates())
gold_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/new_predict/1_gold_learning_asso.tsv")
print len(gold_asso['drug_id'].drop_duplicates())

71
434


In [35]:
indep_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/3_2_indep_drug_se_matrix.tsv", index_col=0)
gold_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/4_2_gold_se_by_indep_matrix.tsv", index_col=0)

In [36]:
print len(indep_se)
indep_se.head(1)

1844


,DB00125,DB00136,DB00153,DB00176,DB00193,DB00195,DB00253,DB00280,DB00283,DB00286,...,DB01195,DB01200,DB01210,DB01254,DB01393,DB01399,DB01410,DB01586,DB01611,DB01620
C0000727,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
print len(gold_se)
gold_se.head(1)

1844


,DB00215,DB01327,DB00537,DB00175,DB00181,DB00257,DB00343,DB01128,DB00678,DB00808,...,DB01033,DB01254,DB01261,DB00323,DB00287,DB00905,DB01088,DB01268,DB09026,DB01590
C0000727,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
gold_dr_id = gold_asso['drug_id'].drop_duplicates()
indep_dr_id = indep_asso['drug_id'].drop_duplicates()

In [41]:
indep_phe_sim_mat = pd.DataFrame(columns = gold_dr_id.tolist(), index = indep_dr_id.tolist())
indep_phe_sim_mat = indep_phe_sim_mat.fillna(0.0)
print len(indep_phe_sim_mat)
indep_phe_sim_mat.head(1)

71


,DB00659,DB01193,DB00316,DB00819,DB00210,DB00640,DB00240,DB00630,DB00346,DB09026,...,DB00162,DB00126,DB02546,DB00682,DB00549,DB00744,DB00246,DB00399,DB00315,DB00909
DB01200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
from sklearn.metrics import jaccard_similarity_score
for i_dr in indep_dr_id:
    for g_dr in gold_dr_id:
        i_se = indep_se[i_dr]
        g_se = gold_se[g_dr]
        
        jac_sim = jaccard_similarity_score(i_se.tolist(), g_se.tolist())
        
        indep_phe_sim_mat.set_value(i_dr, g_dr, jac_sim)

In [45]:
indep_phe_sim_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/2_2_2_indep_drug_gold_drug_se_sim_mat.tsv", sep='\t')

In [15]:
indep_desc = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_2_indep_asso_descriptor.tsv")

In [46]:
indep_drug_feature = indep_desc[['drug_id']+indep_desc.columns.values[3:2279].tolist()]
print indep_drug_feature.shape

(2555, 2277)


In [66]:
indep_drug_feature = indep_drug_feature.drop_duplicates('drug_id')
print indep_drug_feature.shape
indep_drug_feature.head(2)

(73, 2277)


,drug_id,C0000737,C0000786,C0001824,C0002170,C0002453,C0002622,C0002792,C0002871,C0002884,...,C0395005,C0424082,C0549334,C1096460,C0008497,C0581395,C0151964,C1697943,C0567085,C0854954
0,DB00125,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DB00136,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
# 아래 for loop에서 index 지정 에러가 떠서 0으로 통일 시킴.
indep_drug_feature.index = [0 for i in range(len(indep_drug_feature))]

In [70]:
from sklearn.metrics import jaccard_similarity_score
df_jac = pd.DataFrame()
drug_id = df_indep_drug_smiles['DrugBank_ID'].tolist()
for i in drug_id:
    desc1 = indep_drug_feature[indep_drug_feature.drug_id == i].loc[0][1:]
    for j in drug_id:
        desc2 = indep_drug_feature[indep_drug_feature.drug_id == j].loc[0][1:]
        
        jac_sim = jaccard_similarity_score(desc1.tolist(),desc2.tolist())
        
        temp_df = pd.DataFrame([[i, j, jac_sim]], columns = ['DrugBank_ID1','DrugBank_ID2', 'PheSim'])
        df_jac = df_jac.append(temp_df)

In [73]:
df_jac.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_phenotype.tsv", sep='\t', index=False)
df_jac.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_phenotype.tsv", sep='\t', index=False)

In [3]:
#### drug similarities 합치는 과정
drug_ch = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_structure.tsv")
drug_ph = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_phenotype.tsv")
df_drug_sim = pd.merge(left = drug_ch, right= drug_ph, left_on = ['DrugBank_ID1', 'DrugBank_ID2'], right_on = ['DrugBank_ID1', 'DrugBank_ID2'])
df_drug_sim = df_drug_sim.rename(columns = {'Similarity_Score_x' : 'CheSim', 'Similarity_Score_y':'PheSim'})
df_drug_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_whole.tsv", index=False, sep='\t')
df_drug_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_whole.tsv", index=False, sep='\t')